## Connect to DB

In [1]:
%%configure -f
{ "name":"Spark-to-Cosmos_DB_Connector",  
  "conf": {
    "spark.jars.packages": "com.microsoft.azure:azure-cosmosdb-spark_2.4.0_2.11:1.4.0",
    "spark.jars.excludes": "org.scala-lang:scala-reflect"
   }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1574025542235_0005,pyspark,idle,Link,Link,


## Get Data from containers

In [2]:
# Champion Container
readConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "champions", 
}
champ = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1574025542235_0008,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

In [3]:
# match container
matchConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "matches", 
}

match = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**matchConfig).load()

In [4]:
#match filter
match.createOrReplaceTempView("table1")
srmatch=spark.sql("SELECT * from table1 where stats.gameMode='CLASSIC' and stats.gameDuration>420")

In [5]:
#player container
playConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
"Database" : "league_all_data",
"Collection" : "players", 
}

play = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**playConfig).load()

In [10]:
#player filtering/transforming
import pyspark.sql.functions as f
play.createOrReplaceTempView("table2")
play2=spark.sql("SELECT mastery, id from table2")
play2.show(1)

+--------------------+--------------------+
|             mastery|                  id|
+--------------------+--------------------+
|[31119, 218, 2712...|EuXHkripW2Qav3H2l...|
+--------------------+--------------------+
only showing top 1 row

In [12]:
#separates k/v pairs into many columns
expdic2=play2.select(f.col('id').alias('summonerID'),
                     f.col('mastery.*'))
expdic2.show(1)

+--------------------+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+---+---+-----+---+----+----+----+----+----+----+-----+----+---+---+---+----+----+----+----+-----+----+----+----+----+-----+----+----+----+---+----+----+---+----+---+----+----+----+----+-----+----+----+----+----+----+-----+----+-----+-----+----+----+----+-----+-----+---+----+----+----+----+----+----+---+----+----+----+----+----+---+-----+-----+----+-----+----+---+----+----+---+----+---+-----+---+---+----+----+----+---+---+----+----+----+----+----+---+----+----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+----+----+---+----+---+----+----+----+---+----+----+----+----+---+---+----+----+----+----+----+----+----+
|          summonerID|    1| 10| 101| 102| 103| 104| 105| 106| 107|  11| 110| 111| 112| 113|114|115|  117|119|  12| 120| 121| 122| 126| 127|   13| 131|133|134|136|  14| 141| 142| 143|  145|  15| 150| 154| 157|   16| 161| 163| 164| 17|  18|  19|  2|  20|201| 20

## Transforming into final mastery table

In [13]:
from pyspark.sql.functions import array, col, explode, struct, lit
def flat_col(df, by):

    # Filter dtypes and split into column names and type description
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    # Spark SQL supports only homogeneous columns
    assert len(set(dtypes)) == 1, "All columns have to be of the same type"

    # Create and explode an array of (column_name, column_value) structs
    kvs = explode(array([
      struct(lit(c).alias("championId"), col(c).alias("points")) for c in cols
    ])).alias("kvs")

    return df.select(by + [kvs]).select(by + ["kvs.championId", "kvs.points"])

champ_mastery=flat_col(expdic2, ["summonerID"])
champ_mastery.show(5)

+--------------------+----------+------+
|          summonerID|championId|points|
+--------------------+----------+------+
|EuXHkripW2Qav3H2l...|         1| 31119|
|EuXHkripW2Qav3H2l...|        10|   218|
|EuXHkripW2Qav3H2l...|       101|  2712|
|EuXHkripW2Qav3H2l...|       102|  null|
|EuXHkripW2Qav3H2l...|       103|  5543|
+--------------------+----------+------+
only showing top 5 rows

## Champion Table

In [14]:
champ.createOrReplaceTempView("table3")
champ2=spark.sql("SELECT id, name, stats.armorperlevel, stats.attackdamageperlevel, stats.attackspeed, stats.attackrange, stats.spellblockperlevel, stats.hpperlevel FROM table3")
champ2.collect()

[Row(id=u'266', name=u'Aatrox', armorperlevel=3.25, attackdamageperlevel=5.0, attackspeed=0.651, attackrange=175, spellblockperlevel=1.25, hpperlevel=90), Row(id=u'103', name=u'Ahri', armorperlevel=3.5, attackdamageperlevel=3.0, attackspeed=0.668, attackrange=550, spellblockperlevel=0.5, hpperlevel=92), Row(id=u'84', name=u'Akali', armorperlevel=3.5, attackdamageperlevel=3.3, attackspeed=0.625, attackrange=125, spellblockperlevel=1.25, hpperlevel=95), Row(id=u'12', name=u'Alistar', armorperlevel=3.5, attackdamageperlevel=3.62, attackspeed=0.625, attackrange=125, spellblockperlevel=1.25, hpperlevel=106), Row(id=u'32', name=u'Amumu', armorperlevel=3.8, attackdamageperlevel=3.8, attackspeed=0.736, attackrange=125, spellblockperlevel=1.25, hpperlevel=84), Row(id=u'34', name=u'Anivia', armorperlevel=4.0, attackdamageperlevel=3.2, attackspeed=0.625, attackrange=600, spellblockperlevel=0.5, hpperlevel=82), Row(id=u'1', name=u'Annie', armorperlevel=4.0, attackdamageperlevel=2.625, attackspeed=

## Join champ and mastery tables

In [15]:
mast_champ_join = champ_mastery.join(champ2, champ_mastery.championId == champ2.id)
mast_champ_join.show(2)

+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
|          summonerID|championId|points| id|   name|armorperlevel|attackdamageperlevel|attackspeed|attackrange|spellblockperlevel|hpperlevel|
+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
|EuXHkripW2Qav3H2l...|        51|  5333| 51|Caitlyn|          3.5|                2.88|      0.625|        650|               0.5|        91|
|RBQZJQA6Acz3VqBqd...|        51| 30573| 51|Caitlyn|          3.5|                2.88|      0.625|        650|               0.5|        91|
+--------------------+----------+------+---+-------+-------------+--------------------+-----------+-----------+------------------+----------+
only showing top 2 rows

## Save to CosmosDB

In [16]:
writeConfig = {
    "Endpoint": "https://lolpredict.documents.azure.com:443/",
    "Masterkey": "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ==",
    "Database": "league_extraction",
    "Collection": "champion_table",
    "Upsert": "true"
}

# Write to Cosmos DB from the flights DataFrame
champ2.write.format("com.microsoft.azure.cosmosdb.spark").options(
    **writeConfig).save()